In [55]:
import ES.ES_classes as ES
import importlib
importlib.reload(ES)
import torch
import torch.nn as nn

import tensorflow as tf

import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
import numpy as np
import cv2
import pandas as pd

import random

from NeuralGraph import NeuralGraph

In [56]:
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

In [57]:
def split_dataset(x, y, valid_split=.2, test_split=.2):
    classes = np.unique(y)
    train_valid_split = round(len(classes)*(1-(valid_split+test_split)))
    valid_test_split = round(len(classes)*(1-test_split))

    train_classes = set(classes[:train_valid_split])
    valid_classes = set(classes[train_valid_split:valid_test_split])
    test_classes = set(classes[valid_test_split:])

    train = {i:[] for i in train_classes}
    valid = {i:[] for i in valid_classes}
    test = {i:[] for i in test_classes}

    for data, class_ in zip(x, y):
        if class_ in train_classes:
            train[class_].append(data)
        
        elif class_ in valid_classes:
            valid[class_].append(data)

        else:
            test[class_].append(data)

    for class_ in train_classes:
        train[class_] = np.stack(train[class_])

    for class_ in valid_classes:
        valid[class_] = np.stack(valid[class_])

    for class_ in test_classes:
        test[class_] = np.stack(test[class_])

    return train, valid, test

In [58]:
SIZE = 7

# (x, y), (_, _) = tf.keras.datasets.mnist.load_data()
# x = np.stack([cv2.resize(img/255.0, dsize=[SIZE, SIZE]) for img in x]).reshape(-1, SIZE**2)

# _, mnist_valid, mnist_test = split_dataset(x, y, valid_split=.5, test_split=.5)

# (x, y), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
# x = np.stack([cv2.resize(img/255.0, dsize=[SIZE, SIZE]) for img in x]).reshape(-1, SIZE**2)

# _, fashion_valid, fashion_test = split_dataset(x, y, valid_split=.5, test_split=.5)

# emnist = pd.read_csv("emnist-letters-train.csv").values
# x, y = emnist[:, 1:], emnist[:, 0]
# x = np.stack([cv2.resize(cv2.flip(cv2.rotate(img.reshape(28, 28)/255.0, cv2.ROTATE_90_CLOCKWISE), 1), dsize=[SIZE, SIZE]) for img in x]).reshape(-1, SIZE**2)

# emnist_train, emnist_valid, emnist_test = split_dataset(x, y)

In [59]:
# train = [emnist_train]
# valid = [mnist_valid, fashion_valid, emnist_valid]

In [60]:
BATCH_SIZE = 512
TIME = 1
DT = .25

n_classes = 2
device = "cuda" if torch.cuda.is_available() else "cpu"


shape = [SIZE**2, n_classes]
n_nodes = sum(shape)
connections = [(sum(shape[:i])+x, sum(shape[:i+1])+y) for i in range(len(shape)-1) for x in range(shape[i]) for y in range(shape[i+1])]

print(len(connections))

graph = NeuralGraph(n_nodes, SIZE**2, n_classes, connections,
    ch_n=8, ch_e=8, value_init="random", init_value_std=.05, 
    aggregation="mean", use_label=True, clamp_mode="soft", max_value=1e5, 
    device=device)

log = []

98


In [61]:
graph.inp_int = ES.ES_MLP(nn.Sequential(ES.ES_Linear(9, 8))).to(device)
graph.label_int = ES.ES_MLP(nn.Sequential(ES.ES_Linear(9, 8))).to(device)
graph.out_int = ES.ES_MLP(nn.Sequential(ES.ES_Linear(8, 1))).to(device)
graph.messages = nn.ModuleList([ES.ES_MLP(nn.Sequential(
    ES.ES_Linear(24, 32),
    nn.Tanh(),
    ES.ES_Linear(32, 24)
))]).to(device)
graph.updates = nn.ModuleList([ES.ES_MLP(nn.Sequential(
    ES.ES_Linear(24, 32),
    nn.Tanh(),
    ES.ES_Linear(32, 8)
))]).to(device)

ruleset = [
    graph.inp_int,
    graph.label_int,
    graph.out_int,
    graph.messages[0],
    graph.updates[0],
]
for model in ruleset:
    model.generate_epsilons(BATCH_SIZE)

In [62]:
graph.init_vals(batch_size=BATCH_SIZE)
graph.timestep()